# COVID19 - Countries

In [1]:
import os
import ray

In [2]:
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime,timedelta,date
from tabulate import tabulate
from IPython.display import HTML

In [3]:
# Download data
import get_data as gd
LoadData=True

if LoadData:
    gd.get_data()

# SEAIR-D Model Equations

$$\begin{array}{l}\frac{d s}{d t}=-[\beta i(t) + \beta_2 a(t)-\mu] \cdot s(t)\\ 
\frac{d e}{d t}=[\beta i(t) + \beta_2 a(t)] \cdot s(t) -(\sigma+\mu) \cdot e(t)\\ 
\frac{d a}{d t}=\sigma e(t) \cdot (1-p)-(\gamma+\mu) \cdot a(t) \\
\frac{d i}{d t}=\sigma e(t) \cdot p - (\gamma + \sigma_2 + \sigma_3 + \mu) \cdot i(t)\\ 
\frac{d r}{d t}=(b + \sigma_2) \cdot i(t) + \gamma \cdot a(t) - \mu \cdot r(t)\\
\frac{d k}{d t}=(a + \sigma_3 - \mu) \cdot d(t)
\end{array}$$

The sum of all people accounted is equal to population considered:

$$s + e + a + i + r + k = N$$

The derivate is:

$$\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t}+\frac{d k}{d t} = 0$$

The last equation does not need to be solve because:

$$\frac{d k}{d t}=-(\frac{d s}{d t}+\frac{d e}{d t}+\frac{d a}{d t}+\frac{d i}{d t}+\frac{d r}{d t})$$

The sum of all rates are equal to zero! The importance of this equation is that it conservates the rates.


## Parameters

N: the population considered

$\beta$: Effective contact rate [1/min]
    
$\gamma$: Recovery(+Mortality) rate $\gamma=(a+b)$ [1/min]

$a$: mortality of healed  [1/min]

$b$: recovery rate  [1/min]

$\sigma$: is the rate at which individuals move from the exposed to the infectious classes. Its reciprocal ($1/\sigma$) is the average latent (exposed) period.

$\sigma_2$: is the rate at which individuals move from the infectious to the healed classes. Its reciprocal ($1/\sigma_2$) is the average latent (exposed) period

$\sigma_3$: is the rate at which individuals move from the infectious to the dead classes. Its reciprocal ($1/\sigma_3$) is the average latent (exposed) period
    
$p$: is the fraction of the exposed which become symptomatic infectious sub-population.

$(1-p)$: is the fraction of the exposed which becomes asymptomatic infectious sub-population.

## Options: Param.CSV file, changeCSV, Adjust Param, Save CSV, Select Specific Country

In [4]:
from IPython.display import display, HTML, Markdown

changeCSV=False
adjustParam=True
paramSave=False
selectCountries=False

## Different Options to Load Param.CSV file

In [5]:
paramOpt=5
model = "YaboxIC"    

if paramOpt==0:
    paramFile="data/param_optimized_Yabox_HistMin-Copy3.csv"
    version2 = "003"

if paramOpt==1:
    paramFile="data/param_optimized_Yabox_HistMin-Copy4.csv"
    version2 = "003"
    
if paramOpt==2:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "004"
    
if paramOpt==3:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "005"

if paramOpt==4:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "006"

if paramOpt==5:
    paramFile="data/param_optimized_Yabox_HistMin.csv"
    version2 = "007"

In [6]:
dfparam = pd.read_csv(paramFile)
dfparam = dfparam.dropna()
dfparam.e0=0
dfparam.a0=0

display(Markdown("## Original Initial Parameters"))
display(dfparam)

## Original Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC
0,Brazil,03/03/20,200,21305180,0,0,139,129,101,97,0.0830,0.2333
1,China,01/26/20,200,111610854,0,0,200,100,50,50,0.1972,0.0030
2,Italy,02/27/20,200,528128,0,0,266,133,225,6,0.1152,0.2492
3,US,02/20/20,200,98541764,0,0,19,49,122,110,0.0637,0.0168
4,India,03/09/20,200,459980359,0,0,340,61,98,48,0.1100,0.0013


## Modify param.csv file to fit better the data

In [7]:
if changeCSV and (paramOpt==1):
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    dfparam['s0'] = dfparam['s0'].astype(float)
    dfparam['i0'] = dfparam['i0'].astype(float)
    dfparam['d0'] = dfparam['d0'].astype(float)
    dfparam.loc[dfparam.country=='Italy','s0'] = dfparam.loc[dfparam.country=='Italy','s0']*1.8
    dfparam.loc[dfparam.country=='China','s0'] = dfparam.loc[dfparam.country=='China','s0']*1.5
    dfparam.loc[dfparam.country=='India','s0'] = dfparam.loc[dfparam.country=='India','s0']*12/100
    dfparam.loc[dfparam.country=='US','s0'] = dfparam.loc[dfparam.country=='US','s0']*8/10
    dfparam['s0'] = pd.to_numeric(dfparam['s0'],errors='coerce')
    dfparam['i0'] = pd.to_numeric(dfparam['i0'],errors='coerce')
    dfparam['d0'] = pd.to_numeric(dfparam['d0'],errors='coerce')
    

# if adjustParam:
#     sCorrect=[1.38,1.0,1.05,1.5,1.5,1.4,1.1,1.5,1.15,1.55,1.35,1.2,1.35,1.15,1.55,1.45,1.25]
#     dfparam.s0=dfparam.s0.multiply(sCorrect[:len(DRS)], axis=0)

if selectCountries and paramOpt==0:
    countries=['Brazil', 
         'US']

if not selectCountries:
    countries=dfparam.country    

dfparam['s0'] = dfparam['s0'].astype(int)
dfparam['i0'] = dfparam['i0'].astype(int)
dfparam['d0'] = dfparam['d0'].astype(int)

if paramSave:
    dfparam.to_csv("new_"+paramFile)    

display(Markdown("## Modified Initial Parameters"))
display(dfparam)

## Modified Initial Parameters

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC
0,Brazil,03/03/20,200,21305180,0,0,139,129,101,97,0.0830,0.2333
1,China,01/26/20,200,111610854,0,0,200,100,50,50,0.1972,0.0030
2,Italy,02/27/20,200,528128,0,0,266,133,225,6,0.1152,0.2492
3,US,02/20/20,200,98541764,0,0,19,49,122,110,0.0637,0.0168
4,India,03/09/20,200,459980359,0,0,340,61,98,48,0.1100,0.0013


# Solver Loading and Version

In [8]:
%reload_ext autoreload
%autoreload 2

splitUnder=True
underNotif=True
newModel=True

if splitUnder:
    if newModel:
        import LearnerYabox_v5 as Learner 
    else:
        import LearnerYabox_v4 as Learner 
else:
    import LearnerYabox_v3 as Learner


Yabox version:  1.1.0


2020-10-19 20:36:39,854	INFO resource_spec.py:212 -- Starting Ray with 229.98 GiB memory available for workers and up to 72.8 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-19 20:36:40,175	INFO services.py:1170 -- View the Ray dashboard at localhost:8267


In [9]:
allCountries=True
cleanRecovered=False
version="2"

if allCountries:
    display(countries)
else:
    countrySelected="Italy"
    display(countrySelected)

0    Brazil
1     China
2     Italy
3        US
4     India
Name: country, dtype: object

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

# Main Loop

In [ ]:
results=[]
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        current = datetime.strptime(parameters[0], '%m/%d/%y')
        parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
        if country=='Italy':
            endDateStr = "10/15/20"
        else:
            endDate = date.today() + timedelta(days = -2)
            endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
        endDate = datetime.strptime(endDateStr, '%m/%d/%y')
        end_date= datetime.strftime(endDate, '%-m/%-d/%y')   
        f=Learner.Learner.remote(country, *parameters, end_date, 
                                 cleanRecovered, version,underNotif=underNotif)
        result = f.train.remote() 
        results.append(result)
else:
    country = countrySelected 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    current = datetime.strptime(parameters[0], '%m/%d/%y')
    parameters[0] = datetime.strftime(current, '%-m/%-d/%y')
    if country=='Italy':
        endDateStr = "9/1/20"
    else:
        endDate = date.today() + timedelta(days = -2)
        endDateStr = datetime.strftime(endDate, '%-m/%-d/%y')
    endDate = datetime.strptime(endDateStr, '%m/%d/%y')
    end_date= datetime.strftime(endDate, '%-m/%-d/%y')  
    f=Learner.Learner.remote(country, *parameters, end_date, 
                             cleanRecovered, version,underNotif=underNotif)
    result = f.train.remote() 
    results.append(result)

# #execute all the queue with max_runner_cap at a time    
results = ray.get(results)

2020-10-19 20:36:40,994	WARNING worker.py:1090 -- WARNING: 3 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-19 20:36:41,391	WARNING worker.py:1090 -- WARNING: 4 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


  0%|          | 0/21125000.0 [00:00<?, ?it/s]


2020-10-19 20:36:41,788	WARNING worker.py:1090 -- WARNING: 5 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

  0%|          | 61075/21125000.0 [05:34<14:37:36, 400.03it/s]
  0%|          | 26565/21125000.0 [05:34<30:25:24, 192.64it/s]
  0%|          | 41616/21125000.0 [05:35<20:54:55, 280.01it/s]
  0%|          | 24976/21125000.0 [05:34<33:08:04, 176.89it/s]
  0%|          | 61425/21125000.0 [05:35<14:31:30, 402.82it/s]
  0%|          | 28680/21125000.0 [05:34<29:43:03, 197.19it/s]
  0%|          | 41905/21125000.0 [05:36<20:57:42, 279.38it/s]
  0%|          | 26796/21125000.0 [05:35<29:59:07, 195.45it/s]
  0%|          | 61776/21125000.0 [05:35<14:39:10, 399.30it/s]
  0%|          | 25200/21125000.0 [05:35<32:44:05, 179.05it/s]
  0%|          | 28920/21125000.0 [05:35<29:19:47, 199.80it/s]
  0%|          | 42195/21125000.0 [05:37<21:05:46, 277.60it/s]
  0%|          | 27028/21125000.0 [05:37<30:19:45, 193.23it/s]
  0%|          | 62128/21125000.0 [05:36<14:28:54, 404.01it/s]
  0%|          | 25425/21125000.0 [05:36<32:08:26, 182.35it/s]
  0%|          | 29161/21125000.0 [05:37<28:53:12, 202.

  0%|          | 30381/21125000.0 [06:01<28:57:29, 202.35it/s]
  0%|          | 32385/21125000.0 [06:02<27:14:41, 215.05it/s]
  0%|          | 34716/21125000.0 [06:02<25:45:02, 227.50it/s]
  0%|          | 73153/21125000.0 [06:02<13:34:14, 430.91it/s]
  0%|          | 50086/21125000.0 [06:03<18:58:30, 308.52it/s]
  0%|          | 30628/21125000.0 [06:02<28:58:02, 202.28it/s]
  0%|          | 32640/21125000.0 [06:04<27:08:04, 215.92it/s]
  0%|          | 73536/21125000.0 [06:03<13:28:50, 433.77it/s]
  0%|          | 34980/21125000.0 [06:03<25:42:52, 227.82it/s]
  0%|          | 50403/21125000.0 [06:04<18:42:23, 312.94it/s]
  0%|          | 73920/21125000.0 [06:04<13:27:14, 434.63it/s]
  0%|          | 30876/21125000.0 [06:03<28:43:23, 204.00it/s]
  0%|          | 32896/21125000.0 [06:05<27:05:11, 216.30it/s]
  0%|          | 50721/21125000.0 [06:05<18:16:34, 320.31it/s]
  0%|          | 35245/21125000.0 [06:04<25:39:33, 228.31it/s]
  0%|          | 74305/21125000.0 [06:05<13:15:35, 440.

  0%|          | 85491/21125000.0 [06:30<12:39:31, 461.69it/s]
  0%|          | 38781/21125000.0 [06:30<24:34:12, 238.39it/s]
  0%|          | 41328/21125000.0 [06:30<24:27:22, 239.47it/s]
  0%|          | 59340/21125000.0 [06:31<17:14:46, 339.30it/s]
  0%|          | 85905/21125000.0 [06:31<12:33:38, 465.28it/s]
  0%|          | 36585/21125000.0 [06:30<26:40:09, 219.65it/s]
  0%|          | 39060/21125000.0 [06:32<24:32:01, 238.74it/s]
  0%|          | 41616/21125000.0 [06:31<24:34:18, 238.34it/s]
  0%|          | 59685/21125000.0 [06:32<16:59:51, 344.25it/s]
  0%|          | 86320/21125000.0 [06:31<12:32:57, 465.69it/s]
  0%|          | 36856/21125000.0 [06:31<26:19:57, 222.45it/s]
  0%|          | 39340/21125000.0 [06:33<24:27:38, 239.45it/s]
  0%|          | 86736/21125000.0 [06:32<12:27:52, 468.84it/s]
  0%|          | 60031/21125000.0 [06:33<17:05:30, 342.35it/s]
  0%|          | 41905/21125000.0 [06:32<24:26:02, 239.68it/s]
  0%|          | 37128/21125000.0 [06:32<26:07:24, 224.

  0%|          | 45753/21125000.0 [06:58<22:26:42, 260.87it/s]
  0%|          | 99235/21125000.0 [06:58<11:43:39, 498.01it/s]
  0%|          | 48516/21125000.0 [06:58<21:57:50, 266.55it/s]
  0%|          | 43071/21125000.0 [06:58<23:31:01, 249.01it/s]
  0%|          | 69378/21125000.0 [06:59<15:31:59, 376.53it/s]
  0%|          | 46056/21125000.0 [06:59<22:13:12, 263.51it/s]
  0%|          | 99681/21125000.0 [06:59<11:44:38, 497.31it/s]
  0%|          | 69751/21125000.0 [07:00<15:25:31, 379.16it/s]
  0%|          | 48828/21125000.0 [06:59<22:02:02, 265.70it/s]
  0%|          | 43365/21125000.0 [06:59<23:26:48, 249.76it/s]
  0%|          | 100128/21125000.0 [07:00<11:49:27, 493.92it/s]
  0%|          | 46360/21125000.0 [07:00<22:04:07, 265.32it/s]
  0%|          | 70125/21125000.0 [07:01<15:07:30, 386.68it/s]
  0%|          | 43660/21125000.0 [07:00<23:06:37, 253.39it/s]
  0%|          | 49141/21125000.0 [07:00<22:09:54, 264.13it/s]
  0%|          | 100576/21125000.0 [07:01<11:49:15, 49

  1%|          | 113526/21125000.0 [07:26<11:05:02, 526.57it/s]
  0%|          | 56280/21125000.0 [07:26<20:32:31, 284.90it/s]
  0%|          | 80200/21125000.0 [07:27<14:58:37, 390.32it/s]
  0%|          | 50403/21125000.0 [07:26<21:56:57, 266.71it/s]
  0%|          | 53628/21125000.0 [07:27<21:54:34, 267.15it/s]
  1%|          | 114003/21125000.0 [07:27<10:57:31, 532.57it/s]
  0%|          | 56616/21125000.0 [07:27<20:12:41, 289.55it/s]
  0%|          | 80601/21125000.0 [07:28<14:59:46, 389.81it/s]
  1%|          | 114481/21125000.0 [07:28<11:01:46, 529.14it/s]
  0%|          | 50721/21125000.0 [07:27<22:00:28, 266.00it/s]
  0%|          | 53956/21125000.0 [07:28<21:37:44, 270.61it/s]
  0%|          | 81003/21125000.0 [07:29<14:33:35, 401.48it/s]
  1%|          | 114960/21125000.0 [07:29<11:01:31, 529.34it/s]
  0%|          | 56953/21125000.0 [07:28<20:29:15, 285.65it/s]
  0%|          | 54285/21125000.0 [07:29<21:08:06, 276.93it/s]
  0%|          | 51040/21125000.0 [07:28<21:48:19, 

In [ ]:
ray.shutdown()

# Optimized Parameters and Curves Adjust

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'sub', 'subRec', 'subDth']
dfVars = pd.DataFrame(data=results, columns=cols)
if allCountries:
    dfVars['country']=dfparam.country
else:
    dfVars['country']=countrySelected
    display(countrySelected)

In [ ]:
cols=['beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu', 'country','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]

display(Markdown("## Under Notification and Adjusts"))
display(dfVars.iloc[:,12:16])

In [ ]:
cols=['country','beta0', 'beta01', 'startT', 'beta2', 'sigma', 'sigma2', 
      'sigma3', 'gamma', 'b', 'gamma2', 'd', 'mu','sub', 'subRec', 'subDth']
dfVars=dfVars[cols]
dfVars.to_csv('./results/underNotif_vars_'+version2+'.csv', sep=",")

display(Markdown("## Simulation Parameters"))

if newModel:
    cols=['index','country','$\\beta_0$', '$\\beta_{01}$', 'startT', '$\\beta_2$', '$\\sigma$', '$\\sigma_2$', 
          '$\\sigma_3$', 'a', 'b', 'c', 'd', '$\\mu$','sub', 'subRec', 'subDth']
else:
    cols=['index','country','$\\beta_0$', '$\\beta_{01}$', 'startT', '$\\beta_2$', '$\\sigma$', '$\\sigma_2$', 
          '$\\sigma_3$', '$\\gamma$', 'b', '$\\gamma_2$', 'd', '$\\mu$','sub', 'subRec', 'subDth']    
HTML(tabulate(dfVars.iloc[:,:13], headers= cols, floatfmt='.3e', tablefmt='html'))

# Plots

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import covid_plots_v2 as cp

In [ ]:
def loadDataFrame(filename):
    df= pd.read_pickle(filename)
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    df.columns = [c.lower().replace('(', '') for c in df.columns]
    df.columns = [c.lower().replace(')', '') for c in df.columns]
    return df

## Plot Parameters

In [ ]:
#main country for analysis
countryMain = "Brazil"

#select countries for plotting
countries4Plot=['China',
               'US',
               'India',
                countryMain,
               'Italy'
               ]

#Choose here your options
#opt=0 all plots
#opt=1 corona log plot
#opt=2 logistic model prediction
#opt=3 bar plot with growth rate
#opt=4 log plot + bar plot
#opt=5 SEAIR-D Model
opt = 5

#number of cases to start plotting model in log graph - real data = 100
startCase=100

#all Countries for plotting
countries=dfparam.country

In [ ]:
#do not allow the scrolling of the plots

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

## Plot Loop

In [ ]:
#plots one country or all countries  
if opt==5:
    allCountries=True
else:
    allCountries=False
    
if allCountries:
    for country in countries:
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        startCase = query['START'][0]
        startdate = query['start-date'][0]
        predict_range = query['prediction-range'][0]
        #calcula data máxima dos gráficos
        #200 dias é usado como máximo dos cálculos da derivada das mortes
        lastDate='9/22/20'
        maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
        maxDateStr = maxDate.strftime("%-m/%-d/%y")
        cp.covid_plots(country, countries4Plot, startdate, predict_range, \
                        startCase, opt, version, show=True, maxDate=maxDateStr, model=model)
else: 
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    startdate = query['start-date'][0]
    predict_range = query['prediction-range'][0]
    startCase = query['START'][0]
    #calcula data máxima dos gráficos
    #200 dias é usado como máximo dos cálculos da derivada das mortes
    lastDate='9/22/20'
    maxDate= datetime.strptime(lastDate, "%m/%d/%y") + timedelta(days = 200) #"2020-08-31"
    maxDateStr = maxDate.strftime("%-m/%-d/%y")   
    cp.covid_plots(country, countries4Plot,  startdate, predict_range, \
                       startCase, opt, version, show=True, maxDate=maxDateStr, model=model)